In [15]:
import os

import pandas as pd

# Dataset preparation for pretrained open ai model

In [16]:

df = pd.read_csv("../dataset/input/emails_set.csv", index_col=0)
df = df[["text", "is_phishing"]]
df = df.rename(columns={"text": "prompt", "is_phishing": "completion"})
df = df.dropna()

df.to_json("../dataset/input/emails_set.jsonl", orient='records', lines=True)

In [ ]:
!openai tools fine_tunes.prepare_data -f '../dataset/input/emails_set.jsonl' -q

### Merge files made by open ai preparation tool

In [18]:
# Load preprocessed datasets and combine them together for pretrained model as it does not require any prior training

prepared_valid_jsonl = '../dataset/input/emails_set_prepared_valid.jsonl'
prepared_train_jsonl = '../dataset/input/emails_set_prepared_train.jsonl'

if os.path.exists(prepared_valid_jsonl):
    df_first_batch = pd.read_json(prepared_valid_jsonl, lines=True)
    df_second_batch = pd.read_json(prepared_train_jsonl, lines=True)
    df = pd.concat([df_first_batch, df_second_batch], ignore_index=True)
    df.to_json('../dataset/input/emails_set_batch.jsonl', orient='records', lines=True)

    os.remove(prepared_valid_jsonl)
    os.remove(prepared_train_jsonl)


## Fine-tuned model dataset preparation

In [19]:
# Calculate the split index
split_idx = int(len(df) * 0.7)

# Split into training and testing sets
train_data = df.iloc[:split_idx]
test_data = df.iloc[split_idx:]

train_data.to_json("../dataset/input/emails_set_train.jsonl", orient='records', lines=True)
test_data.to_json("../dataset/input/emails_set_test.jsonl", orient='records', lines=True)